In [17]:
balance_sheet

,Treasury Shares Number,Ordinary Shares Number,Share Issued,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,...,Duefrom Related Parties Current,Notes Receivable,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Equivalents,Cash Financial
2023-12-31,0.0,25932070992.0,25932070992.0,956257900000.0,3406755700000.0,4357098500000.0,1251227800000.0,3406755700000.0,28681800000.0,3429522400000.0,...,696300000.0,201313900000.0,NaN,NaN,NaN,1714803200000.0,249375400000.0,1465427800000.0,12326200000.0,1453101600000.0
2022-12-31,NaN,25930380458.0,25930380458.0,888174400000.0,2877020500000.0,3761430000000.0,1066333100000.0,2877020500000.0,29764100000.0,2903019700000.0,...,1653000000.0,229755900000.0,NaN,NaN,NaN,1586500100000.0,243686000000.0,1342814100000.0,13522700000.0,1329291400000.0
2021-12-31,NaN,25930380458.0,25930380458.0,753631900000.0,2122438100000.0,2882127500000.0,848720100000.0,2122438100000.0,20764200000.0,2149259800000.0,...,776800000.0,197586100000.0,197586100000.0,-347000000.0,197933100000.0,1204913700000.0,139923500000.0,1064990200000.0,6182100000.0,1058808100000.0
2020-12-31,NaN,25930380458.0,25930380458.0,367792300000.0,1809043200000.0,2182043000000.0,460286900000.0,1809043200000.0,20560600000.0,1834811300000.0,...,608700000.0,145480300000.0,145480300000.0,-246600000.0,145726900000.0,799893200000.0,139722600000.0,660170600000.0,6590000000.0,653580600000.0
2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,135978000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

# Function to fetch financial data
def fetch_financial_data(ticker):
    """
    Fetches financial data for a given ticker using yfinance.

    Parameters:
    ticker (str): Stock ticker symbol.

    Returns:
    dict: Dictionary containing financial statements (income, balance, cashflow) and info.
    """
    company = yf.Ticker(ticker)
    financial_data = {
        'income_statement': company.financials.T,
        'balance_sheet': company.balance_sheet.T,
        'cash_flow': company.cashflow.T,
        'info': company.info
    }
    return financial_data


In [3]:
# Function to extract and clean FCF data
def extract_fcf(cash_flow, fcf_column='Free Cash Flow'):
    """
    Extracts Free Cash Flow (FCF) from the cash flow statement.

    Parameters:
    cash_flow (DataFrame): Cash flow statement data.
    fcf_column (str): Column name for Free Cash Flow. Default is 'Free Cash Flow'.

    Returns:
    Series: Cleaned Free Cash Flow data.
    """
    if fcf_column in cash_flow.columns:
        fcf = cash_flow[fcf_column].dropna()
        return fcf
    else:
        raise ValueError(f"{fcf_column} column is missing from the cash flow statement.")


In [4]:

# Function to calculate CAGR (Compound Annual Growth Rate)
def calculate_cagr(historical_fcf):
    """
    Calculates the Compound Annual Growth Rate (CAGR) based on historical FCF data.

    Parameters:
    historical_fcf (Series): Historical Free Cash Flow data.

    Returns:
    float: CAGR value.
    """
    start_value = historical_fcf.iloc[0]
    end_value = historical_fcf.iloc[-1]
    periods = len(historical_fcf) - 1
    
    if start_value <= 0 or end_value <= 0:
        raise ValueError("FCF values must be positive for CAGR calculation.")
    
    cagr = (end_value / start_value)**(1 / periods) - 1
    return cagr


In [5]:

# Function to calculate historical average growth rate
def calculate_average_growth_rate(historical_fcf):
    """
    Calculates the average annual growth rate based on historical FCF data.

    Parameters:
    historical_fcf (Series): Historical Free Cash Flow data.

    Returns:
    float: Average annual growth rate.
    """
    growth_rates = historical_fcf.pct_change().dropna()
    avg_growth_rate = growth_rates.mean()
    return avg_growth_rate


In [6]:

# Function to forecast FCF using calculated growth rate
def forecast_fcf(historical_fcf, growth_rate, years=5):
    """
    Forecasts future FCF based on historical FCF and a calculated growth rate.

    Parameters:
    historical_fcf (Series): Historical Free Cash Flow.
    growth_rate (float): Calculated growth rate for forecasting.
    years (int): Number of years to forecast. Default is 5 years.

    Returns:
    list: Forecasted FCF for the specified years.
    """
    last_fcf = historical_fcf[-1]
    fcf_forecast = [last_fcf * (1 + growth_rate)**i for i in range(1, years + 1)]
    return fcf_forecast


In [7]:

# Function to calculate the terminal value
def calculate_terminal_value(last_fcf, terminal_growth_rate=0.02, discount_rate=0.10):
    """
    Calculates the terminal value using the Gordon Growth Model.

    Parameters:
    last_fcf (float): Last projected Free Cash Flow.
    terminal_growth_rate (float): Growth rate for terminal value. Default is 2%.
    discount_rate (float): Discount rate (WACC). Default is 10%.

    Returns:
    float: Terminal Value.
    """
    terminal_value = last_fcf * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    return terminal_value


In [8]:

# Function to calculate the present value of future cash flows
def discount_cash_flows(fcf_forecast, terminal_value, discount_rate):
    """
    Calculates the present value of forecasted FCFs and terminal value.

    Parameters:
    fcf_forecast (list): Forecasted FCF values.
    terminal_value (float): Terminal value.
    discount_rate (float): Discount rate (WACC).

    Returns:
    float: Total discounted cash flow value.
    """
    discounted_fcf = [fcf / (1 + discount_rate)**i for i, fcf in enumerate(fcf_forecast, 1)]
    discounted_terminal_value = terminal_value / (1 + discount_rate)**len(fcf_forecast)
    return sum(discounted_fcf) + discounted_terminal_value


In [18]:

# Main DCF function with automatic growth rate calculation
def dcf_analysis(ticker, terminal_growth_rate=0.02, discount_rate=0.10, growth_rate_method='cagr'):
    """
    Performs DCF analysis for a given stock ticker using calculated growth rate based on historical FCF data.

    Parameters:
    ticker (str): Stock ticker symbol.
    terminal_growth_rate (float): Growth rate for terminal value. Default is 2%.
    discount_rate (float): Discount rate (WACC). Default is 10%.
    growth_rate_method (str): Method to calculate growth rate ('cagr' or 'average'). Default is 'cagr'.

    Returns:
    float: Intrinsic value per share.
    """
    # Fetch financial data
    financial_data = fetch_financial_data(ticker)
    cash_flow = financial_data['cash_flow']
    
    # Extract Free Cash Flow (FCF)
    historical_fcf = extract_fcf(cash_flow)
    
    # Calculate growth rate based on method
    if growth_rate_method == 'cagr':
        growth_rate = calculate_cagr(historical_fcf)
    elif growth_rate_method == 'average':
        growth_rate = calculate_average_growth_rate(historical_fcf)
    else:
        raise ValueError("Invalid growth rate calculation method. Choose 'cagr' or 'average'.")
    
    print(f"Calculated Growth Rate: {growth_rate:.4f}")
    
    # Forecast future FCF
    fcf_forecast = forecast_fcf(historical_fcf, growth_rate)
    
    # Calculate terminal value
    terminal_value = calculate_terminal_value(fcf_forecast[-1], terminal_growth_rate, discount_rate)
    
    # Discount FCFs and terminal value
    total_pv = discount_cash_flows(fcf_forecast, terminal_value, discount_rate)
    
    # Calculate enterprise value
    enterprise_value = total_pv
    
    # Calculate equity value (assuming net debt)
    balance_sheet = financial_data['balance_sheet']
    net_debt = balance_sheet['Total Debt'].iloc[-1] - balance_sheet['Cash Cash Equivalents And Short Term Investments'].iloc[-1]
    equity_value = enterprise_value - net_debt
    
    # Intrinsic value per share
    shares_outstanding = financial_data['info']['sharesOutstanding'] / 1e6  # in million
    intrinsic_value_per_share = equity_value / shares_outstanding
    
    return intrinsic_value_per_share


In [19]:

# Run DCF analysis for a given stock ticker using calculated growth rate
if __name__ == '__main__':
    ticker = 'TSM'  # Example ticker
    intrinsic_value = dcf_analysis(ticker, growth_rate_method='cagr')
    print(f"Intrinsic Value per Share for {ticker}: ${intrinsic_value:.2f}")


Calculated Growth Rate: 0.0220
Intrinsic Value per Share for TSM: $nan


C:\Users\slove\AppData\Local\Temp\ipykernel_14864\2278281519.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_fcf = historical_fcf[-1]
